In [33]:
import numpy as np
import pandas as pd
import pickle

In [47]:
!python data_preprocess.py sample-data/spa spa

In [48]:
!python data_preprocess.py sample-data/eng eng

In [64]:
!python data_preprocess.py sample-data/ben ben

In [67]:
!python data_preprocess.py sample-data/grc grc

In [69]:
!python data_preprocess.py sample-data/ell ell

In [71]:
!python utils.py sample-data/spa spa
!python utils.py sample-data/eng eng
!python utils.py sample-data/ben ben
!python utils.py sample-data/grc grc
!python utils.py sample-data/ell ell

In [51]:
!python em.py sample-data/spa spa
!python em.py sample-data/eng eng
!python em.py sample-data/ben ben
!python em.py sample-data/grc grc
!python em.py sample-data/ell ell

In [55]:
!python em.py sample-data/spa spa

[0.20548751772894755, 0.3124891971517955, 0.1503043770305493, 0.2955595108544126, 0.03615939723429485]
[0.22423172155800603, 0.325070318871512, 0.15060811764179868, 0.29130708215025725, 0.008782759778426234]
[0.238736321817739, 0.3305897341479557, 0.1476369073532463, 0.28094533163358815, 0.0020917050474709444]
[0.25213962685490565, 0.33406681208747147, 0.14388091029094022, 0.26941704062053695, 0.0004956101461457119]
[0.2652542157784308, 0.33677408891074156, 0.13995139721376532, 0.25790305649786893, 0.00011724159919334063]
[0.2782688579630014, 0.33902848625565196, 0.13599833575731918, 0.24667660597525007, 2.771404877737027e-05]
[0.29121461641136653, 0.34091511851466694, 0.1320630546192937, 0.23580066269002886, 6.547764643842594e-06]
[0.30408434669772644, 0.3424638518522063, 0.12816110806717257, 0.2252891470866996, 1.5462961951023658e-06]
[0.31686274551643656, 0.343691634036812, 0.12430148988840263, 0.21514376554039585, 3.6501795294628594e-07]
[0.32953380907865804, 0.3446125265665924, 0.

[0.8093361648768683, 0.176589536100391, 0.00850355190023457, 0.005570747122506141, 1.3716870713786858e-50]
[0.8124503217716117, 0.17412223833733015, 0.008156501002742397, 0.005270938888315836, 3.214674981959397e-51]
[0.8155047582055089, 0.17168457596355893, 0.007823476452624444, 0.00498718937830762, 7.533807380890884e-52]
[0.818500940993366, 0.1692764898453821, 0.007503923226194008, 0.004718645935058065, 1.765580800086393e-52]
[0.8214402924654208, 0.16689789986535528, 0.007197307232035017, 0.004464500437188985, 4.137676583987494e-53]
[0.8243241918349812, 0.1645487065060955, 0.006903114612895272, 0.0042239870460279725, 9.696646215154347e-54]
[0.8271539765401084, 0.16222879233822712, 0.006620851062489394, 0.0039963800591750085, 2.2723897432633345e-54]
[0.8299309435594922, 0.15993802341658664, 0.0063500411576667085, 0.0037809918662544506, 5.3252565848545725e-55]
[0.8326563507040747, 0.157676250586806, 0.006090227706249754, 0.0035771710028693732, 1.247943381140248e-55]
[0.8353314178738724,

[0.9400628178273344, 0.05948469242046412, 0.00036611129294884386, 8.637845925277666e-05, 7.47452489671579e-98]
[0.9409477734045589, 0.05861947630915301, 0.0003510454012898052, 8.170488499831507e-05, 1.7514098018882377e-98]
[0.94181928755855, 0.05776682878775298, 0.0003365994698532954, 7.728418384361503e-05, 4.103856633408509e-99]
[0.9426775791935779, 0.05692657013434726, 0.00032274799700121105, 7.310267507366714e-05, 9.616052210772594e-100]
[0.9435228630487317, 0.056098523004469936, 0.00030946652893878805, 6.914741785959413e-05, 2.2532100442021657e-100]
[0.9443553498010797, 0.055282512410877054, 0.0002967316167698406, 6.540617127345125e-05, 5.279670064694322e-101]
[0.9451752461649431, 0.054478365703291995, 0.00028452077530182027, 6.18673564630791e-05, 1.2371208691959764e-101]
[0.9459827549890097, 0.0536859125465994, 0.0002728124435255711, 5.8520020865211484e-05, 2.898796525538005e-102]
[0.9467780753494613, 0.05290498489947018, 0.00026158594671587415, 5.5353804352709875e-05, 6.792405189

[0.9795541533347686, 0.02042730749984857, 1.704869382716047e-05, 1.4904715555497532e-06, 7.413385205768969e-144]
[0.9798514544599631, 0.020130788137927628, 1.6347531287301677e-05, 1.4098708218666898e-06, 1.7371643232865244e-144]
[0.9801444068576058, 0.019838584300971407, 1.567521205760786e-05, 1.333629365209755e-06, 4.0706660602687505e-145]
[0.9804330749757119, 0.01955063296358638, 1.5030549323238571e-05, 1.2615113785978266e-06, 9.538724137059254e-146]
[0.9807175222784804, 0.019266872022590628, 1.4412405116363155e-05, 1.1932938126755266e-06, 2.2351945918941323e-146]
[0.9809978112620293, 0.01898724028398016, 1.381968830508839e-05, 1.1287656853900693e-06, 5.237700016214856e-147]
[0.9812740034703249, 0.01871167744958072, 1.3251352665319998e-05, 1.067727429032474e-06, 1.2273435261050655e-147]
[0.981546159510671, 0.0184401241040241, 1.2706395032244104e-05, 1.0099902726003242e-06, 2.8760198642789366e-148]
[0.981814339068827, 0.01817252170198801, 1.2183853527484616e-05, 9.553756575913608e-07,

[0.9929654289049284, 0.007033749800129687, 7.955189901543387e-07, 2.5775951508561506e-08, 7.373971037691298e-190]
[0.9930673440121461, 0.006931868785981903, 7.628192158454947e-07, 2.4382656049771335e-08, 1.7279781330240723e-190]
[0.9931677800056831, 0.006831465465919558, 7.314637183368897e-07, 2.306467902568498e-08, 4.049255362540915e-191]
[0.9932667584293449, 0.006732518355479794, 7.013972267892179e-07, 2.1817948555391616e-08, 9.488819758247116e-192]
Smoothed lambdas:
[0.9932667584293449, 0.006732518355479794, 7.013972267892179e-07, 2.1817948555391616e-08, 9.488819758247116e-192]


In [49]:
with open("sample-data/spa/spacount_sum.pickle", 'rb') as outp:
    counting_dict = pickle.load(outp)


counting_dict['char3_p_s_t'][('rse','Ε','das','V-PTCP-PST-FEM-PL')]

237

In [15]:
import numpy as np
result = {0: 1.1181753789488595, 1: 0.5566080288678394, 2: 0.4718269778030734, 3: 0.48716683119447185, 4: 1.0, 5: 0.1395076201641266, 6: 0.20941558441558442}

names = ['id','data']
formats = ['f8','f8']
dtype = dict(names = names, formats=formats)
array = np.array(list(result.items()), dtype=dtype)

aa =(repr(array))

aa[:]

"array([(0., 1.11817538), (1., 0.55660803), (2., 0.47182698),\n       (3., 0.48716683), (4., 1.        ), (5., 0.13950762),\n       (6., 0.20941558)], dtype=[('id', '<f8'), ('data', '<f8')])"

In [157]:
ἱερόδουλος	τοῦ ῑ̔εροδούλου	N-GEN-SG	Ε	ἱερόδουλος->τοῦ	Ε
ἱερόδουλος	τοῦ ῑ̔εροδούλου	N-GEN-SG	Ε	ἱερόδουλος->το	υ
ἱερόδουλος	τοῦ ῑ̔εροδούλου	N-GEN-SG	τοῦ	ἱερόδουλος->ῑ̔εροδούλου	Ε

'aa'

In [192]:
def reconstruct_inf(lemma, inf, trip):
    eps = '\u0395'
    sc = trip[1].split("->")
    if sc[0]!= eps and (sc[0] != lemma[-len(sc[0]):]):
        return 0
    add_p = trip[0] + lemma if trip[0] != eps else lemma
    add_sc1 = add_p[:-len(sc[0])] if sc[0] == lemma[-len(sc[0]):] else add_p
    add_sc2 = add_sc1 + sc[1] if sc[1] != eps else add_sc1
    add_s = add_sc2 + trip[2] if trip[2] != eps else add_sc2
    if add_s == inf:
        return 1
    else:
        return 0


my_list =[['Ε', 've->Ε', 'd'],['Ε', 'o->Ε', 'ed'], ['Ε', 'e->Ε', 'ed'], ['Ε', 'Ε->Ε', 'd']]

lemma = "abacinate"
inf="abacinated"
          
# indices = [i for i, x in enumerate(my_list) if a == "".join(x.split("->"))]
# list(map(lambda x: x if reconstruct_inf(lemma,inf,x) else 0, my_list))
[x for x in my_list if reconstruct_inf(lemma,inf,x)]

[['Ε', 'e->Ε', 'ed'], ['Ε', 'Ε->Ε', 'd']]

In [142]:
x="a->b"
"".join(x.split("->"))

'ab'

In [255]:
inp = 'X'
oup = 'ab cd ef gh'
oups = ['ab','cd','ef','gh']

In [262]:
for ind_o, oup_s in enumerate(oups):
#     print(ind_o, oups[0:ind_o],oups[ind_o])
    pref = ' '.join(oups[0:ind_o])
    pref = "E" if len(pref)==0 else pref
    oup_r = oup.replace(pref,"")
    print(pref,"|", oup_r)

E | ab cd ef gh
ab |  cd ef gh
ab cd |  ef gh
ab cd ef |  gh


In [ ]:
def alignment(inputs, outputs, poss, characters):
	temp = [(''.join(inputs[i]), ''.join(outputs[i])) for i in range(len(outputs))]
	aligned = align.Aligner(temp).alignedpairs
	vocab = list(characters)
	try:
		vocab.remove(u" ")
	except:
		pass

	triplets = []
	for k, item in enumerate(aligned):
		inp, oup, pos = ''.join(inputs[k]), ''.join(outputs[k]), '-'.join(poss[k])
		tx_set = [inp, oup, pos]
		i, o = item[0], item[1]
		inp_len, oup_len, al_len = len(inp), len(oup), len(i)
		lss, mb, stc = seqMatch(inp, oup)
		eps = '\u0395'
		prefix_flag = False
		tag = ''.join(list(zip(*stc))[0])
		print(tag)
		if len(inp.split(" ")) < len(oup.split(" ")):
			oups = oup.split(" ")
			for ind_o, oup_s in enumerate(oups):
				pref = ' '.join(oups[0:ind_o])
				pref =  pref if len(pref)>0 else eps
				triplet = make_triplet(tx_set, pref, "{}->{}".format(inp, oup_s), eps)
				triplets.append(triplet)
				triplet = make_triplet(tx_set, pref, "{}->{}".format(inp, oup_s[:-1]), oup[-1:])
				triplets.append(triplet)
				triplet = make_triplet(tx_set, pref, "{}->{}".format(inp, oup_s[:-2]), oup[-2:])


		print("_________")

	return triplets